In [1]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

Found existing installation: kai 0.1.1.dev4+g9e2ca63
Uninstalling kai-0.1.1.dev4+g9e2ca63:
  Successfully uninstalled kai-0.1.1.dev4+g9e2ca63
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/konveyor/kai.git (to revision main) to /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-77_s8aju
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-77_s8aju
  Resolved https://github.com/konveyor/kai.git to commit 9e2ca639af9c7077077e20a4da4144aa64eede6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kai: filename=kai-0.1.1.dev4+g9e2ca63-py3-none-any.whl size=4425694 sha256=978ee71620396852f485991e1fefba4047ea368793d679fd5d33512545f5bfa0
  Stored in directory: /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/

In [10]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
load_dotenv() 

# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-3.5-turbo"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



In [12]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

In [16]:
analysis_rule=""""\
    
  ruleID: spring-security-replace-authenticationmanagerbuilder-00001
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  category: mandatory
  effort: 2
  labels:
    - konveyor.io/source=spring-boot2
    - konveyor.io/target=spring-boot3+
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    ```
  when:
    java.referenced:
      location: METHOD
      pattern: configure(org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder) *
    """

In [17]:
prompt = f"""
```java
{before_code}
```

{analysis_rule}

Can you me a fix for this issue and update the source code?

"""

In [18]:

response = await rendered_llm_call(prompt)

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    
    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}

```
In the above code snippet, the `configure(AuthenticationManagerBuilder)` method has been removed as it is discouraged in Spring Security 5.7. Instead, a `PasswordEncoder` bean has been defined using the `BCryptPasswordEncoder`. This is the recommended approach for handling authentication in Spring Security 5.7 and above.